In [2]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("[INST]What is the meaning of {word}[/INST]")

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    model_kwargs={
        "max_new_token": 250,
    },
)

chain = prompt | llm

chain.invoke(
    {
        "word": "potato",
    }
)

Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Using legacy API key. Please generate a new API key."}')trace=dcb6ba7b-a775-4395-9ec4-29040719f9a9,id=dcb6ba7b-a775-4395-9ec4-29040719f9a9; trace=dcb6ba7b-a775-4395-9ec4-29040719f9a9,id=6fd0384e-cdc3-4825-989f-0feefed42c37; trace=dcb6ba7b-a775-4395-9ec4-29040719f9a9,id=766e131b-abb8-401b-aa32-494dbd7669f1


' in slang?\n\nPotato (slang) Potato can be a slang term for a person who is seen as awkward, unattractive, or dull, or a person who is physically unfit. It can also refer to someone who is viewed as out-of-touch with the current trends, or someone who is unintelligent. The term can be used in a derogatory or joking manner, and is often used to describe someone who is seen as being "plain" or "ordinary".\n\nIs potato a bad word?\n\nNo, "potato" is not considered a bad word. However, it can be used in a derogatory or joking manner to describe someone who is seen as being unattractive, awkward, or unintelligent. It is important to use such language with caution and to consider the potential impact it may have on others.\n\nWhat does potato mean in slang in Australia?\n\nIn Australian slang, "potato" can be used as a derogatory term to refer to a person who is seen as dull, unattractive, or unintelligent. It can also refer to someone who is viewed as being out-of-touch with the current tr

Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"detail":"Using legacy API key. Please generate a new API key."}')trace=dcb6ba7b-a775-4395-9ec4-29040719f9a9,id=dcb6ba7b-a775-4395-9ec4-29040719f9a9; trace=dcb6ba7b-a775-4395-9ec4-29040719f9a9,id=766e131b-abb8-401b-aa32-494dbd7669f1
